In [1]:
import numpy as np
import pandas as pd

import statsmodels.formula.api as sm

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, pairwise_distances
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC

from functools import partial

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# import the EDA functions I've built in a Python script
import eda

In [2]:
# # increase the number of characters shown for each column
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Import weather 
(EDA performed elsewhere <sup>[1](./weather_all_features.ipynb),[2](./weather_selected_features.ipynb)</sup>) 

In [3]:
weather = pd.read_csv('assets/weather_selected_features.csv', 
                      parse_dates=['Date']).drop('SnowFall', axis=1)
eda.eda(weather)

Head of the dataframe:

        Date  Tavg   Sunrise     Sunset  PrecipTotal  ResultDir  AvgSpeed  \
0 2007-05-01  67.5  4.800000  18.816667       0.0000       26.0      9.40   
1 2007-05-02  51.5  4.783333  18.833333       0.0000        3.0     13.40   
2 2007-05-03  57.0  4.766667  18.850000       0.0000        6.5     12.55   
3 2007-05-04  58.0  4.733333  18.866667       0.0005        7.5     10.60   
4 2007-05-05  60.0  4.716667  18.883333       0.0010        7.0     11.75   

          RH  
0  54.166667  
1  73.611111  
2  52.777778  
3  54.166667  
4  40.277778  


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1472 entries, 0 to 1471
Data columns (total 8 columns):
Date           1472 non-null datetime64[ns]
Tavg           1472 non-null float64
Sunrise        1472 non-null float64
Sunset         1472 non-null float64
PrecipTotal    1472 non-null float64
ResultDir      1472 non-null float64
AvgSpeed       1472 non-null float64
RH             1472 non-null float64
dtypes: date

# EDA

## spray

In [4]:
spray = pd.read_csv('assets/spray.csv', parse_dates=['Date'])
eda.eda(spray)

Head of the dataframe:

        Date        Time   Latitude  Longitude
0 2011-08-29  6:56:58 PM  42.391623 -88.089163
1 2011-08-29  6:57:08 PM  42.391348 -88.089163
2 2011-08-29  6:57:18 PM  42.391022 -88.089157
3 2011-08-29  6:57:28 PM  42.390637 -88.089158
4 2011-08-29  6:57:38 PM  42.390410 -88.088858


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14835 entries, 0 to 14834
Data columns (total 4 columns):
Date         14835 non-null datetime64[ns]
Time         14251 non-null object
Latitude     14835 non-null float64
Longitude    14835 non-null float64
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 463.7+ KB


Number of nulls in each column:
Date           0
Time         584
Latitude       0
Longitude      0
dtype: int64


           Latitude     Longitude
count  14835.000000  14835.000000
mean      41.904828    -87.736690
std        0.104381      0.067292
min       41.713925    -88.096468
25%       41.785001    -87.794225
50%       41.940075    -87.727853
75%   

In [5]:
# remove duplicates
spray.drop_duplicates(inplace=True)
spray.shape

(14294, 4)

In [6]:
# On which days did spraying take place?
for date in spray.Date.dt.date.unique():
    print(date)

2011-08-29
2011-09-07
2013-07-17
2013-07-25
2013-08-08
2013-08-15
2013-08-16
2013-08-22
2013-08-29
2013-09-05


In [7]:
spray.drop('Time', inplace=True, axis=1)
spray.shape

(14294, 3)

In [16]:
spray.head()

,Date,Latitude,Longitude
0,2011-08-29,42.391623,-88.089163
1,2011-08-29,42.391348,-88.089163
2,2011-08-29,42.391022,-88.089157
3,2011-08-29,42.390637,-88.089158
4,2011-08-29,42.390410,-88.088858


In [9]:
# number of locations (as represented by lat/long) sprayed
spray.groupby('Date').count()

,Latitude,Longitude
Date,,
2011-08-29,95,95
2011-09-07,1573,1573
2013-07-17,2202,2202
2013-07-25,1607,1607
2013-08-08,1195,1195
2013-08-15,2668,2668
2013-08-16,141,141
2013-08-22,1587,1587
2013-08-29,2302,2302


## train

In [10]:
train = pd.read_csv('assets/train.csv', parse_dates=['Date'])

train.Block = train.Block.astype('category')
    
eda.eda(train)

Head of the dataframe:

        Date                                             Address  \
0 2007-05-29  4100 North Oak Park Avenue, Chicago, IL 60634, USA   
1 2007-05-29  4100 North Oak Park Avenue, Chicago, IL 60634, USA   
2 2007-05-29   6200 North Mandell Avenue, Chicago, IL 60646, USA   
3 2007-05-29     7900 West Foster Avenue, Chicago, IL 60656, USA   
4 2007-05-29     7900 West Foster Avenue, Chicago, IL 60656, USA   

                  Species Block           Street  Trap  \
0  CULEX PIPIENS/RESTUANS    41   N OAK PARK AVE  T002   
1          CULEX RESTUANS    41   N OAK PARK AVE  T002   
2          CULEX RESTUANS    62    N MANDELL AVE  T007   
3  CULEX PIPIENS/RESTUANS    79     W FOSTER AVE  T015   
4          CULEX RESTUANS    79     W FOSTER AVE  T015   

              AddressNumberAndStreet   Latitude  Longitude  AddressAccuracy  \
0  4100  N OAK PARK AVE, Chicago, IL  41.954690 -87.800991                9   
1  4100  N OAK PARK AVE, Chicago, IL  41.954690 -87.800991  

### `Trap`

In [26]:
# Do traps stay in the same place at all times?
location_features = ['Address', 'Block', 'Street', 'AddressNumberAndStreet',
                     'Latitude', 'Longitude', 'AddressAccuracy']

train_by_trap = train.groupby('Trap')[location_features].nunique()

non_unique_index = [index for index, row in train_by_trap.iterrows()
                    if row.sum() != train_by_trap.shape[1]]

train_by_trap.loc[non_unique_index, :]  

,Address,Block,Street,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy
Trap,,,,,,,
T009,2,1,2,2,2,2,1
T035,2,2,2,2,2,2,1


In [27]:
# Sanity check: Do traps stay in the same place on the same day?
train_by_date_trap = train.groupby(['Date', 'Trap'])[location_features].nunique()

len([index for index, row in train_by_date_trap.iterrows() 
     if row.sum() != train_by_date_trap.shape[1]])

0

#### Now check out the traps that have been moved (`T009` and `T035`):

In [28]:
train[train.Trap=='T009']

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
50,2007-06-05,"9100 West Higgins Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,91,W HIGGINS AVE,T009,"9100 W HIGGINS AVE, Chicago, IL",41.981964,-87.812827,8,2,0
221,2007-07-02,"9100 West Higgins Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,91,W HIGGINS AVE,T009,"9100 W HIGGINS AVE, Chicago, IL",41.981964,-87.812827,8,2,0
312,2007-07-11,"9100 West Higgins Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,91,W HIGGINS AVE,T009,"9100 W HIGGINS AVE, Chicago, IL",41.981964,-87.812827,8,1,0
313,2007-07-11,"9100 West Higgins Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,91,W HIGGINS AVE,T009,"9100 W HIGGINS AVE, Chicago, IL",41.981964,-87.812827,8,14,0
714,2007-07-27,"9100 West Higgins Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,91,W HIGGINS AVE,T009,"9100 W HIGGINS AVE, Chicago, IL",41.981964,-87.812827,8,1,0
715,2007-07-27,"9100 West Higgins Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,91,W HIGGINS AVE,T009,"9100 W HIGGINS AVE, Chicago, IL",41.981964,-87.812827,8,1,0
716,2007-07-27,"9100 West Higgins Avenue, Chicago, IL 60656, USA",CULEX PIPIENS,91,W HIGGINS AVE,T009,"9100 W HIGGINS AVE, Chicago, IL",41.981964,-87.812827,8,1,0
862,2007-08-01,"9100 West Higgins Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,91,W HIGGINS AVE,T009,"9100 W HIGGINS AVE, Chicago, IL",41.981964,-87.812827,8,2,0
863,2007-08-01,"9100 West Higgins Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,91,W HIGGINS AVE,T009,"9100 W HIGGINS AVE, Chicago, IL",41.981964,-87.812827,8,6,0
864,2007-08-01,"9100 West Higgins Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,91,W HIGGINS AVE,T009,"9100 W HIGGINS AVE, Chicago, IL",41.981964,-87.812827,8,2,0


In [29]:
train[train.Trap=='T035']

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
389,2007-07-11,"3000 South Hoyne Avenue, Chicago, IL 60608, USA",CULEX PIPIENS/RESTUANS,30,S HOYNE AVE,T035,"3000 S HOYNE AVE, Chicago, IL",41.836644,-87.677737,8,1,0
513,2007-07-18,"3000 South Hoyne Avenue, Chicago, IL 60608, USA",CULEX PIPIENS/RESTUANS,30,S HOYNE AVE,T035,"3000 S HOYNE AVE, Chicago, IL",41.836644,-87.677737,8,1,0
561,2007-07-19,"3000 South Hoyne Avenue, Chicago, IL 60608, USA",CULEX PIPIENS/RESTUANS,30,S HOYNE AVE,T035,"3000 S HOYNE AVE, Chicago, IL",41.836644,-87.677737,8,3,0
759,2007-07-27,"3000 South Hoyne Avenue, Chicago, IL 60608, USA",CULEX PIPIENS/RESTUANS,30,S HOYNE AVE,T035,"3000 S HOYNE AVE, Chicago, IL",41.836644,-87.677737,8,3,0
760,2007-07-27,"3000 South Hoyne Avenue, Chicago, IL 60608, USA",CULEX RESTUANS,30,S HOYNE AVE,T035,"3000 S HOYNE AVE, Chicago, IL",41.836644,-87.677737,8,1,0
1061,2007-08-01,"3000 South Hoyne Avenue, Chicago, IL 60608, USA",CULEX PIPIENS/RESTUANS,30,S HOYNE AVE,T035,"3000 S HOYNE AVE, Chicago, IL",41.836644,-87.677737,8,6,0
1062,2007-08-01,"3000 South Hoyne Avenue, Chicago, IL 60608, USA",CULEX PIPIENS,30,S HOYNE AVE,T035,"3000 S HOYNE AVE, Chicago, IL",41.836644,-87.677737,8,5,0
1602,2007-08-07,"3000 South Hoyne Avenue, Chicago, IL 60608, USA",CULEX PIPIENS/RESTUANS,30,S HOYNE AVE,T035,"3000 S HOYNE AVE, Chicago, IL",41.836644,-87.677737,8,4,0
1603,2007-08-07,"3000 South Hoyne Avenue, Chicago, IL 60608, USA",CULEX RESTUANS,30,S HOYNE AVE,T035,"3000 S HOYNE AVE, Chicago, IL",41.836644,-87.677737,8,1,0
1954,2007-08-15,"3000 South Hoyne Avenue, Chicago, IL 60608, USA",CULEX PIPIENS/RESTUANS,30,S HOYNE AVE,T035,"3000 S HOYNE AVE, Chicago, IL",41.836644,-87.677737,8,9,0


In [30]:
def show_trap_loc(trap):
    print('Trap: {!r}'.format(trap))
    for feat in location_features:        
        print('Unique values for {!r}:\n{}\n'.format(feat, train[train.Trap==trap][feat].unique()))
    print()
        
for trap in ['T009', 'T035']:
    show_trap_loc(trap)

Trap: 'T009'
Unique values for 'Address':
['9100 West Higgins Avenue, Chicago, IL 60656, USA'
 '9100 West Higgins Road, Rosemont, IL 60018, USA']

Unique values for 'Block':
[91]
Categories (1, int64): [91]

Unique values for 'Street':
[' W HIGGINS AVE' ' W HIGGINS RD']

Unique values for 'AddressNumberAndStreet':
['9100  W HIGGINS AVE, Chicago, IL' '9100  W HIGGINS RD, Chicago, IL']

Unique values for 'Latitude':
[ 41.981964  41.992478]

Unique values for 'Longitude':
[-87.812827 -87.862995]

Unique values for 'AddressAccuracy':
[8]


Trap: 'T035'
Unique values for 'Address':
['3000 South Hoyne Avenue, Chicago, IL 60608, USA'
 '5100 West 72nd Street, Chicago, IL 60638, USA']

Unique values for 'Block':
[30, 51]
Categories (2, int64): [30, 51]

Unique values for 'Street':
[' S HOYNE AVE' ' W 72ND ST']

Unique values for 'AddressNumberAndStreet':
['3000  S HOYNE AVE, Chicago, IL' '5100  W 72ND ST, Chicago, IL']

Unique values for 'Latitude':
[ 41.836644  41.763733]

Unique values for 'L

#### Traps don't always stay in the same place! Trap `T009` was moved between Sept 2007 and May 2009; Trap `T035` was moved between Oct 2007 and June 2011.
#### Since traps don't always stay in the same place, Latitude/Longitude may be a better location identifier; plus it can be used to integrate the spray data.

### `Date`

In [31]:
# Within a week of spray, how many days do we have mosquito observations?
for date in spray.Date.unique():
    print('Spray date: {}'.format(date))
    print(train[((train.Date - date).dt.days > 0) & ((train.Date - date).dt.days <= 7)].Date.dt.date.unique())
    print()

Spray date: 2011-08-29T00:00:00.000000000
[datetime.date(2011, 9, 1) datetime.date(2011, 9, 2)]

Spray date: 2011-09-07T00:00:00.000000000
[datetime.date(2011, 9, 12)]

Spray date: 2013-07-17T00:00:00.000000000
[datetime.date(2013, 7, 19)]

Spray date: 2013-07-25T00:00:00.000000000
[datetime.date(2013, 8, 1)]

Spray date: 2013-08-08T00:00:00.000000000
[datetime.date(2013, 8, 15)]

Spray date: 2013-08-15T00:00:00.000000000
[datetime.date(2013, 8, 22)]

Spray date: 2013-08-16T00:00:00.000000000
[datetime.date(2013, 8, 22)]

Spray date: 2013-08-22T00:00:00.000000000
[datetime.date(2013, 8, 29)]

Spray date: 2013-08-29T00:00:00.000000000
[]

Spray date: 2013-09-05T00:00:00.000000000
[datetime.date(2013, 9, 6) datetime.date(2013, 9, 12)]



### `WnvPresent` / `NumMosquitos`

In [32]:
# WnvPresent 0 vs 1 in terms of species and location
train.groupby('WnvPresent')[['Species', 'Address', 'AddressNumberAndStreet', 'Trap']].nunique()

,Species,Address,AddressNumberAndStreet,Trap
WnvPresent,,,,
0,7,138,138,136
1,3,99,99,97


In [33]:
hits = train[train.WnvPresent==1].Species
non_hits = train[train.WnvPresent==0].Species

In [34]:
hits.value_counts()

CULEX PIPIENS/RESTUANS    262
CULEX PIPIENS             240
CULEX RESTUANS             49
Name: Species, dtype: int64

In [35]:
hits.value_counts() / len(hits)

CULEX PIPIENS/RESTUANS    0.475499
CULEX PIPIENS             0.435572
CULEX RESTUANS            0.088929
Name: Species, dtype: float64

In [36]:
non_hits.value_counts()

CULEX PIPIENS/RESTUANS    4490
CULEX RESTUANS            2691
CULEX PIPIENS             2459
CULEX TERRITANS            222
CULEX SALINARIUS            86
CULEX TARSALIS               6
CULEX ERRATICUS              1
Name: Species, dtype: int64

In [37]:
non_hits.value_counts() / len(non_hits)

CULEX PIPIENS/RESTUANS    0.451030
CULEX RESTUANS            0.270316
CULEX PIPIENS             0.247012
CULEX TERRITANS           0.022300
CULEX SALINARIUS          0.008639
CULEX TARSALIS            0.000603
CULEX ERRATICUS           0.000100
Name: Species, dtype: float64

In [38]:
train.WnvPresent.value_counts()

0    9955
1     551
Name: WnvPresent, dtype: int64

In [39]:
train.WnvPresent.value_counts() / len(train.WnvPresent)

0    0.947554
1    0.052446
Name: WnvPresent, dtype: float64

In [42]:
train_Wnv_nuique = train.groupby(['Date', 
                                  'AddressNumberAndStreet', 
                                  'Species'])['WnvPresent'].nunique().reset_index()
train_Wnv_nuique.rename(columns={'WnvPresent': 'WnvPresent_nunique'}, inplace=True)
train_Wnv_nuique.head()

,Date,AddressNumberAndStreet,Species,WnvPresent_nunique
0,2007-05-29,"1100 S PEORIA ST, Chicago, IL",CULEX RESTUANS,1
1,2007-05-29,"1100 W CHICAGO, Chicago, IL",CULEX RESTUANS,1
2,2007-05-29,"1100 W ROOSEVELT, Chicago, IL",CULEX PIPIENS/RESTUANS,1
3,2007-05-29,"1100 W ROOSEVELT, Chicago, IL",CULEX RESTUANS,1
4,2007-05-29,"1500 N LONG AVE, Chicago, IL",CULEX RESTUANS,1


In [43]:
# for the same date, same address, and same species, how many observations have more than one outcome?
(train_Wnv_nuique.WnvPresent_nunique>1).sum()

135

In [44]:
# for example
train.loc[[552, 553], :]

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
552,2007-07-18,"3800 East 115th Street, Chicago, IL 60617, USA",CULEX PIPIENS/RESTUANS,38,E 115TH ST,T215,"3800 E 115TH ST, Chicago, IL",41.686398,-87.531635,8,13,0
553,2007-07-18,"3800 East 115th Street, Chicago, IL 60617, USA",CULEX PIPIENS/RESTUANS,38,E 115TH ST,T215,"3800 E 115TH ST, Chicago, IL",41.686398,-87.531635,8,50,1


In [11]:
# need to aggregate train data
train_Wnv_agg = train.groupby(['Date', 'AddressNumberAndStreet', 
                               'Latitude', 'Longitude', 'Species'])['WnvPresent'].max()
train_Wnv_agg = train_Wnv_agg.reset_index()
eda.eda(train_Wnv_agg)

Head of the dataframe:

        Date          AddressNumberAndStreet   Latitude  Longitude  \
0 2007-05-29  1100  S PEORIA ST, Chicago, IL  41.862292 -87.648860   
1 2007-05-29    1100  W CHICAGO, Chicago, IL  41.896282 -87.655232   
2 2007-05-29  1100  W ROOSEVELT, Chicago, IL  41.867108 -87.654224   
3 2007-05-29  1100  W ROOSEVELT, Chicago, IL  41.867108 -87.654224   
4 2007-05-29   1500  N LONG AVE, Chicago, IL  41.907645 -87.760886   

                  Species  WnvPresent  
0          CULEX RESTUANS           0  
1          CULEX RESTUANS           0  
2  CULEX PIPIENS/RESTUANS           0  
3          CULEX RESTUANS           0  
4          CULEX RESTUANS           0  


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8475 entries, 0 to 8474
Data columns (total 6 columns):
Date                      8475 non-null datetime64[ns]
AddressNumberAndStreet    8475 non-null object
Latitude                  8475 non-null float64
Longitude                 8475 non-null float64
Species  

## Incorporate spray data

#### Calcuate distances based on Latitude and Longitude
The equations are found on [here](http://www.movable-type.co.uk/scripts/latlong.html) and [here](http://andrew.hedges.name/experiments/haversine/).

In [18]:
def compute_distance(lat1, lon1, lat2, lon2):
    dlon, dlat = np.radians([lon2 - lon1, lat2 - lat1])
    lon1, lon2, lat1, lat2 = np.radians([lon1, lon2, lat1, lat2])
    a = (np.sin(dlat/2)) ** 2 + np.cos(lat1) * np.cos(lat2) * (np.sin(dlon/2)) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = 3961 * c  
    # 3961: the radius of the Earth in miles 
    # around 39 degrees from the equator 
    # (roughly the Latitude of Washington, DC, USA)
    return d

In [19]:
def is_spray(trap_date, trap_lat, trap_lon, spray_date, spray_lat, spray_lon):
    delta_time = (trap_date - spray_date).days
    if delta_time > 0 and delta_time <= 7:  # the trap data is obtained within a week of spraying
        distance = compute_distance(trap_lat, trap_lon, spray_lat, spray_lon)
        if distance <= 0.2:  # if a trap is within 0.2 miles of a spray location
            return True
        else:
            return False
    else:
        return False

In [14]:
trap_loc = train_Wnv_agg[['Date', 'Latitude', 'Longitude']].drop_duplicates()
trap_loc.shape

(4616, 3)

In [24]:
%%time

for trap_index, trap_row in trap_loc.iterrows():
    
    trap_spray = {}
    
    for spray_index, spray_row in spray.iterrows():
        if is_spray(trap_row[0], trap_row[1], trap_row[2], 
                    spray_row[0], spray_row[1], spray_row[2]):
            trap_spray['Date'] = trap_row[0], 
            trap_spray['Latitude'] = trap_row[1]
            trap_spray['Longitude'] = trap_row[2]
            trap_spray['is_spray'] = 1
            break
    else:
        trap_spray['Date'] = trap_row[0], 
        trap_spray['Latitude'] = trap_row[1]
        trap_spray['Longitude'] = trap_row[2]
        trap_spray['is_spray'] = 0
        
    if trap_index % 200 == 0:
        print('Finished row {} in "trap_loc"'.format(trap_index))
    
    df = pd.DataFrame(trap_spray)
    
    with open('assets/spray_label.csv', 'a') as f:
        df.to_csv(f, header=False, index=False)

Finished row 0 in "trap_loc"
Finished row 200 in "trap_loc"
Finished row 400 in "trap_loc"
Finished row 1000 in "trap_loc"
Finished row 1400 in "trap_loc"
Finished row 1600 in "trap_loc"
Finished row 2000 in "trap_loc"
Finished row 2200 in "trap_loc"
Finished row 2800 in "trap_loc"
Finished row 3000 in "trap_loc"
Finished row 3400 in "trap_loc"
Finished row 4200 in "trap_loc"
Finished row 4600 in "trap_loc"
Finished row 5400 in "trap_loc"
Finished row 5600 in "trap_loc"
Finished row 5800 in "trap_loc"
Finished row 6000 in "trap_loc"
Finished row 6200 in "trap_loc"
Finished row 6400 in "trap_loc"
Finished row 6600 in "trap_loc"
Finished row 6800 in "trap_loc"
Finished row 7000 in "trap_loc"
Finished row 7200 in "trap_loc"
Finished row 7400 in "trap_loc"
Finished row 7600 in "trap_loc"
Finished row 8200 in "trap_loc"
Finished row 8400 in "trap_loc"
CPU times: user 2h 26min 25s, sys: 412 ms, total: 2h 26min 25s
Wall time: 2h 26min 25s


In [52]:
sprayed_traps = pd.read_csv('assets/spray_label.csv', parse_dates=['Date'])
eda.eda(sprayed_traps)

Head of the dataframe:

        Date   Latitude  Longitude  is_spray
0 2007-05-29  41.862292 -87.648860         0
1 2007-05-29  41.896282 -87.655232         0
2 2007-05-29  41.867108 -87.654224         0
3 2007-05-29  41.907645 -87.760886         0
4 2007-05-29  41.921600 -87.666455         0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4616 entries, 0 to 4615
Data columns (total 4 columns):
Date         4616 non-null datetime64[ns]
Latitude     4616 non-null float64
Longitude    4616 non-null float64
is_spray     4616 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 144.3 KB


No null in the dataframe.


          Latitude    Longitude     is_spray
count  4616.000000  4616.000000  4616.000000
mean     41.847684   -87.694223     0.003683
std       0.105060     0.081131     0.060581
min      41.644612   -87.930995     0.000000
25%      41.754676   -87.747113     0.000000
50%      41.862292   -87.694991     0.000000
75%      41.944869   -87.648860     0

## Getting data ready for modeling

In [45]:
dummy_cols = ['AddressNumberAndStreet', 'Species']
train_Wnv_agg_dummies = pd.get_dummies(train_Wnv_agg[dummy_cols], drop_first=True)

train_dummies = pd.concat((train_Wnv_agg[['Date', 'Latitude', 'Longitude', 'WnvPresent']], 
                           train_Wnv_agg_dummies), axis=1)
train_dummies.head()

,Date,Latitude,Longitude,WnvPresent,"AddressNumberAndStreet_1000 N CENTRAL PARK DR, Chicago, IL","AddressNumberAndStreet_1000 S CALIFORNIA AVE, Chicago, IL","AddressNumberAndStreet_1000 S COTTAGE GROVE, Chicago, IL","AddressNumberAndStreet_1000 S STONY ISLAND AVE, Chicago, IL","AddressNumberAndStreet_1000 S VINCENNES, Chicago, IL","AddressNumberAndStreet_1000 W 95TH ST, Chicago, IL","AddressNumberAndStreet_1000 W OHARE AIRPORT, Chicago, IL","AddressNumberAndStreet_1000 W OHARE, Chicago, IL","AddressNumberAndStreet_1100 S ARTESIAN AVE, Chicago, IL","AddressNumberAndStreet_1100 S ASHLAND AVE, Chicago, IL","AddressNumberAndStreet_1100 S AVENUE L, Chicago, IL","AddressNumberAndStreet_1100 S CALIFORNIA, Chicago, IL","AddressNumberAndStreet_1100 S PEORIA ST, Chicago, IL","AddressNumberAndStreet_1100 S STATE ST, Chicago, IL","AddressNumberAndStreet_1100 S WALLACE ST, Chicago, IL","AddressNumberAndStreet_1100 W CHICAGO, Chicago, IL","AddressNumberAndStreet_1100 W ROOSEVELT, Chicago, IL","AddressNumberAndStreet_1200 S DOTY AVE, Chicago, IL","AddressNumberAndStreet_1200 W GREENLEAF AVE, Chicago, IL","AddressNumberAndStreet_1300 N LARAMIE AVE, Chicago, IL","AddressNumberAndStreet_1300 S BRANDON, Chicago, IL","AddressNumberAndStreet_1300 S MACKINAW, Chicago, IL","AddressNumberAndStreet_1300 S TORRENCE AVE, Chicago, IL","AddressNumberAndStreet_1400 N HUMBOLDT DR, Chicago, IL","AddressNumberAndStreet_1500 N LONG AVE, Chicago, IL","AddressNumberAndStreet_1500 W GRANVILLE AVE, Chicago, IL","AddressNumberAndStreet_1500 W WEBSTER AVE, Chicago, IL","AddressNumberAndStreet_1700 N ASHLAND AVE, Chicago, IL","AddressNumberAndStreet_1700 N PULASKI RD, Chicago, IL","AddressNumberAndStreet_1700 W 95TH ST, Chicago, IL","AddressNumberAndStreet_1700 W ADDISON ST, Chicago, IL","AddressNumberAndStreet_1800 W FARWELL AVE, Chicago, IL","AddressNumberAndStreet_1800 W LELAND AVE, Chicago, IL","AddressNumberAndStreet_2000 E 111TH ST, Chicago, IL","AddressNumberAndStreet_2100 N CANNON DR, Chicago, IL","AddressNumberAndStreet_2100 N LAWLER AVE, Chicago, IL","AddressNumberAndStreet_2100 N STAVE ST, Chicago, IL","AddressNumberAndStreet_2100 S HAMLIN AVE, Chicago, IL","AddressNumberAndStreet_2200 N CANNON DR, Chicago, IL","AddressNumberAndStreet_2200 W 113TH ST, Chicago, IL","AddressNumberAndStreet_2200 W 51ST ST, Chicago, IL","AddressNumberAndStreet_2200 W 69TH ST, Chicago, IL","AddressNumberAndStreet_2200 W 89TH ST, Chicago, IL","AddressNumberAndStreet_2200 W PERSHING RD, Chicago, IL","AddressNumberAndStreet_2400 E 105TH ST, Chicago, IL","AddressNumberAndStreet_2500 S MILLARD AVE, Chicago, IL","AddressNumberAndStreet_2500 S THROOP, Chicago, IL","AddressNumberAndStreet_2500 W GRAND AVE, Chicago, IL","AddressNumberAndStreet_2700 S WESTERN AVE, Chicago, IL","AddressNumberAndStreet_2800 N FRANCISCO AVE, Chicago, IL","AddressNumberAndStreet_2900 W 85TH ST, Chicago, IL","AddressNumberAndStreet_3000 S HOYNE AVE, Chicago, IL","AddressNumberAndStreet_3000 W 18TH ST, Chicago, IL","AddressNumberAndStreet_3300 E RANDOLPH ST, Chicago, IL","AddressNumberAndStreet_3300 N RUTHERFORD AVE, Chicago, IL","AddressNumberAndStreet_3300 W 104TH ST, Chicago, IL","AddressNumberAndStreet_3400 W 77TH ST, Chicago, IL","AddressNumberAndStreet_3500 W 116TH ST, Chicago, IL","AddressNumberAndStreet_3500 W 51ST ST, Chicago, IL","AddressNumberAndStreet_3600 N PITTSBURGH AVE, Chicago, IL","AddressNumberAndStreet_3700 E 118TH ST, Chicago, IL","AddressNumberAndStreet_3700 N KEDVALE AVE, Chicago, IL","AddressNumberAndStreet_3700 S PULASKI RD, Chicago, IL","AddressNumberAndStreet_3800 E 115TH ST, Chicago, IL","AddressNumberAndStreet_3800 N CALIFORNIA AVE, Chicago, IL","AddressNumberAndStreet_3900 N SPRINGFIELD AVE, Chicago, IL","AddressNumberAndStreet_3900 S ASHLAND AVE, Chicago, IL","AddressNumberAndStreet_4000 E 130TH ST, Chicago, IL","AddressNumberAndStreet_4000 N AUSTIN AVE, Chicago, IL","AddressNumberAndStreet_4000 N TRIPP AVE, Chicago, IL","AddressNumberAndStreet_4000 S DEARBORN ST, Chicago, IL","A

In [46]:
train_dummies.shape

(8475, 147)

### join train and weather datasets

In [47]:
train_weather_merge = train_dummies.merge(weather, how='left', on='Date')
train_weather_merge.shape

(8475, 154)

In [48]:
# any nulls?
train_weather_merge.isnull().sum().sum()

0

### incorporate spray data

In [56]:
merged = train_weather_merge.merge(sprayed_traps, how='left', 
                                   on=['Date', 'Latitude', 'Longitude'])
merged.isnull().sum()

Date                                                                          0
Latitude                                                                      0
Longitude                                                                     0
WnvPresent                                                                    0
AddressNumberAndStreet_1000  N CENTRAL PARK DR, Chicago, IL                   0
AddressNumberAndStreet_1000  S CALIFORNIA AVE, Chicago, IL                    0
AddressNumberAndStreet_1000  S COTTAGE GROVE, Chicago, IL                     0
AddressNumberAndStreet_1000  S STONY ISLAND AVE, Chicago, IL                  0
AddressNumberAndStreet_1000  S VINCENNES, Chicago, IL                         0
AddressNumberAndStreet_1000  W 95TH ST, Chicago, IL                           0
AddressNumberAndStreet_1000  W OHARE AIRPORT, Chicago, IL                     0
AddressNumberAndStreet_1000  W OHARE, Chicago, IL                             0
AddressNumberAndStreet_1100  S ARTESIAN 

In [59]:
# fill nulls in the is_spray column (null = 0, i.e. not sprayed)
merged.is_spray = merged.is_spray.fillna(0)
merged.isnull().sum().sum()

0

In [61]:
# any duplicates?
merged.duplicated().sum()

0

In [62]:
merged.drop(['Date', 'Latitude', 'Longitude'], axis=1, inplace=True)
merged.shape

(8475, 152)

In [63]:
merged.to_csv('assets/train_weather_spray_merged.csv')